In [1]:
import pandas as pd 
import requests
import argparse
import numpy as np
import time

In [2]:

client_id = "ceac365a-b813-450d-af8e-34137f7874df"  # Frost client-id (brukernavn i Basic Auth)
url = "https://frost.met.no/observations/v0.jsonld"

params = {
    "sources": "SN18700",                          # Oslo - Blindern
    "elements": "air_temperature,wind_speed",      # velg noen elementer
    "referencetime": "2020-01-01/2020-01-02",      # et kort intervall
    "timeresolutions": "PT1H"                      # timeverdier (valgfritt)
}

r = requests.get(url, params=params, auth=(client_id, ""), timeout=30)

print("Status:", r.status_code)
print("URL:", r.url)
print("Content-Type:", r.headers.get("content-type"))
print("Første 500 tegn av responsen:\n", r.text[:500])

r.raise_for_status()  # kaster exception hvis noe er galt


Status: 200
URL: https://frost.met.no/observations/v0.jsonld?sources=SN18700&elements=air_temperature%2Cwind_speed&referencetime=2020-01-01%2F2020-01-02&timeresolutions=PT1H
Content-Type: application/json
Første 500 tegn av responsen:
 {
  "@context" : "https://frost.met.no/schema",
  "@type" : "ObservationResponse",
  "apiVersion" : "v0",
  "license" : "https://creativecommons.org/licenses/by/3.0/no/",
  "createdAt" : "2025-09-18T12:15:51Z",
  "queryTime" : 0.227,
  "currentItemCount" : 24,
  "itemsPerPage" : 24,
  "offset" : 0,
  "totalItemCount" : 24,
  "currentLink" : "https://frost.met.no/observations/v0.jsonld?sources=SN18700&elements=air_temperature%2Cwind_speed&referencetime=2020-01-01%2F2020-01-02&timeresolutions=PT1H


In [3]:
df = pd.read_csv("../data/training_data.csv")

# lag datetime (lokal Oslo-tid antatt) og konverter til UTC
df["dt_local"] = pd.to_datetime(df["date"]) + pd.to_timedelta(df["hour"], unit="h")
df["dt_local"] = df["dt_local"].dt.tz_localize("Europe/Oslo", nonexistent="shift_forward", ambiguous="NaT")
df["dt_utc"]   = df["dt_local"].dt.tz_convert("UTC")

df.head()

,airport_group,date,hour,target,feat_season,feat_sched_flights_cnt,feat_sched_concurrence,target2,dt_local,dt_utc
0,A,2018-01-01,0,0,winter,0,0,0,2018-01-01 00:00:00+01:00,2017-12-31 23:00:00+00:00
1,B,2018-01-01,0,0,winter,0,0,0,2018-01-01 00:00:00+01:00,2017-12-31 23:00:00+00:00
2,C,2018-01-01,0,0,winter,0,0,0,2018-01-01 00:00:00+01:00,2017-12-31 23:00:00+00:00
3,D,2018-01-01,0,0,winter,0,0,0,2018-01-01 00:00:00+01:00,2017-12-31 23:00:00+00:00
4,E,2018-01-01,0,0,winter,0,0,0,2018-01-01 00:00:00+01:00,2017-12-31 23:00:00+00:00


In [4]:
import requests

CLIENT_ID = "ceac365a-b813-450d-af8e-34137f7874df"  # din
url = "https://frost.met.no/observations/v0.jsonld"
params = {
    "sources": "SN18700",
    "elements": "air_temperature",
    "referencetime": "2024-10-01/2024-10-02",  # kun 1 dag
}

r = requests.get(url, params=params, auth=(CLIENT_ID, ""))
print("Status:", r.status_code)
print("URL:", r.url)
print("Body (first 300 chars):", r.text[:300])


Status: 200
URL: https://frost.met.no/observations/v0.jsonld?sources=SN18700&elements=air_temperature&referencetime=2024-10-01%2F2024-10-02
Body (first 300 chars): {
  "@context" : "https://frost.met.no/schema",
  "@type" : "ObservationResponse",
  "apiVersion" : "v0",
  "license" : "https://creativecommons.org/licenses/by/3.0/no/",
  "createdAt" : "2025-09-18T12:16:03Z",
  "queryTime" : 0.662,
  "currentItemCount" : 1440,
  "itemsPerPage" : 1440,
  "offset" :


In [5]:

CLIENT_ID = "ceac365a-b813-450d-af8e-34137f7874df"
url = "https://frost.met.no/observations/v0.jsonld"

params = {
    "sources": "SN18700",  # Blindern
    "elements": "air_temperature,wind_speed,wind_from_direction,precipitation_amount",
    "referencetime": "2024-10-01/2024-10-31",
    "timeresolutions": "PT1H"
}

r = requests.get(url, params=params, auth=(CLIENT_ID, ""))
print("Status:", r.status_code)
r.raise_for_status()

rows = []
for e in r.json().get("data", []):
    ts = e.get("referenceTime")
    rec = {"timestamp_utc": pd.to_datetime(ts, utc=True)}
    for o in e.get("observations", []):
        if "value" in o:
            rec[o["elementId"]] = o["value"]
    rows.append(rec)

df = pd.DataFrame(rows).sort_values("timestamp_utc")
df.rename(columns={
    "air_temperature": "temp_C",
    "wind_speed": "wind_ms",
    "wind_from_direction": "wind_dir_deg",
    "precipitation_amount": "precip_mm"
}, inplace=True)

print(df.head(10))
print(df.tail(10))


Status: 200
              timestamp_utc  temp_C  wind_ms  wind_dir_deg
0 2024-10-01 00:00:00+00:00     9.1      1.1            27
1 2024-10-01 01:00:00+00:00     8.3      2.1            27
2 2024-10-01 02:00:00+00:00     7.4      1.9            17
3 2024-10-01 03:00:00+00:00     7.3      3.6            30
4 2024-10-01 04:00:00+00:00     6.9      3.0            36
5 2024-10-01 05:00:00+00:00     6.6      2.6            32
6 2024-10-01 06:00:00+00:00     7.2      2.9            39
7 2024-10-01 07:00:00+00:00     9.0      3.1            31
8 2024-10-01 08:00:00+00:00     9.5      2.4            38
9 2024-10-01 09:00:00+00:00    10.4      2.5            30
                timestamp_utc  temp_C  wind_ms  wind_dir_deg
710 2024-10-30 14:00:00+00:00    12.8      4.4           262
711 2024-10-30 15:00:00+00:00    11.5      2.8           264
712 2024-10-30 16:00:00+00:00     9.7      1.6           259
713 2024-10-30 17:00:00+00:00     7.9      1.6           259
714 2024-10-30 18:00:00+00:00     